<a href="https://colab.research.google.com/github/rpujala/machine_learning/blob/main/Demand_Spike_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Demand Spike Detection (Bullwhip Prevention)

* The demand planning team observed sudden spikes in customer orders that were not driven by real consumption, but by panic ordering, promotions, or upstream miscommunication.

* These artificial spikes propagate upstream, causing overproduction, excess inventory, and the bullwhip effect

* The team wants a machine learning system that can flay abnormal demand spikes in near real time allowing planners to pause replenishment, investigate root causes, or smooth orders before they amplify across the supply chain

In [ ]:
import pandas as pd
import numpy as np

np.random.seed(42)

n_samples = 7000
n_skus = 50

data = {
    'sku_id': np.random.choice([f"SKU_{i}" for i in range(n_skus)], n_samples),
    'region': np.random.choice(['NA', 'EU', 'APAC'], n_samples),
    'avg_demand_7d': np.random.uniform(20, 200, n_samples),
    'avg_demand_28d': np.random.uniform(20, 180, n_samples),
    'std_demand_28d': np.random.uniform(5, 40, n_samples),
    'promo_active': np.random.choice([0, 1], n_samples, p=[0.7, 0.3]),
    'holiday_week': np.random.choice([0, 1], n_samples, p=[0.85, 0.15]),
    'inventory_position': np.random.randint(0, 800, n_samples)
}

df = pd.DataFrame(data=data)

# current demand with noise
df['demand_t'] = df['avg_demand_7d'] + np.random.normal(0, df['std_demand_28d']).clip(0)

df.head()

,sku_id,region,avg_demand_7d,avg_demand_28d,std_demand_28d,promo_active,holiday_week,inventory_position,demand_t
0,SKU_38,EU,167.296738,168.290617,32.741872,1,0,90,194.777723
1,SKU_28,NA,135.353745,132.910023,17.603818,0,0,561,135.353745
2,SKU_14,NA,75.871657,44.314670,16.317941,0,0,288,75.871657
3,SKU_42,NA,138.100007,82.958827,37.961360,1,0,91,138.100007
4,SKU_7,APAC,57.883115,29.003191,6.457075,0,0,746,59.236771


In [ ]:
df['spike_flag'] = (
    df['demand_t'] > (df['avg_demand_28d'] + 2 * df['std_demand_28d'])
).astype(int)

In [ ]:
df.head()

,sku_id,region,avg_demand_7d,avg_demand_28d,std_demand_28d,promo_active,holiday_week,inventory_position,demand_t,spike_flag
0,SKU_38,EU,167.296738,168.290617,32.741872,1,0,90,194.777723,0
1,SKU_28,NA,135.353745,132.910023,17.603818,0,0,561,135.353745,0
2,SKU_14,NA,75.871657,44.314670,16.317941,0,0,288,75.871657,0
3,SKU_42,NA,138.100007,82.958827,37.961360,1,0,91,138.100007,0
4,SKU_7,APAC,57.883115,29.003191,6.457075,0,0,746,59.236771,1


# Check Blank values

In [ ]:
import numpy as np

df.replace('', np.nan, inplace=True)
df.isna().sum()

sku_id                0
region                0
avg_demand_7d         0
avg_demand_28d        0
std_demand_28d        0
promo_active          0
holiday_week          0
inventory_position    0
demand_t              0
spike_flag            0
dtype: int64

# Feature / Target Split

In [ ]:
df.head()

,sku_id,region,avg_demand_7d,avg_demand_28d,std_demand_28d,promo_active,holiday_week,inventory_position,demand_t,spike_flag
0,SKU_38,EU,167.296738,168.290617,32.741872,1,0,90,194.777723,0
1,SKU_28,NA,135.353745,132.910023,17.603818,0,0,561,135.353745,0
2,SKU_14,NA,75.871657,44.314670,16.317941,0,0,288,75.871657,0
3,SKU_42,NA,138.100007,82.958827,37.961360,1,0,91,138.100007,0
4,SKU_7,APAC,57.883115,29.003191,6.457075,0,0,746,59.236771,1


In [ ]:
X = df.drop(columns=['sku_id', 'spike_flag'], axis=1)
y = df['spike_flag']

In [ ]:
X[:2]

,region,avg_demand_7d,avg_demand_28d,std_demand_28d,promo_active,holiday_week,inventory_position,demand_t
0,EU,167.296738,168.290617,32.741872,1,0,90,194.777723
1,NA,135.353745,132.910023,17.603818,0,0,561,135.353745


In [ ]:
y[:2]

0    0
1    0
Name: spike_flag, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

np.random.seed(42)

X_train, X_test, y_train, y_test = train_test_split(X,
                                                   y,
                                                   test_size=0.2,
                                                   random_state=42,
                                                   stratify=y)

In [ ]:
X_train.shape, y_train.shape

((5600, 8), (5600,))

In [ ]:
X_test.shape, y_test.shape

((1400, 8), (1400,))

# Preprocessing Pipeline (Production-Grade)

In [ ]:
X[:2]

,region,avg_demand_7d,avg_demand_28d,std_demand_28d,promo_active,holiday_week,inventory_position,demand_t
0,EU,167.296738,168.290617,32.741872,1,0,90,194.777723
1,NA,135.353745,132.910023,17.603818,0,0,561,135.353745


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

In [ ]:
num_features = [
    'avg_demand_7d',
    'avg_demand_28d',
    'std_demand_28d',
    'inventory_position',
    'demand_t'
]

cat_features = [
    'region',
    'promo_active',
    'holiday_week'
]

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('one_hot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num_pipeline', num_pipeline, num_features),
    ('cat_pipeline', cat_pipeline, cat_features)
])

preprocessor

,transformers,"[('num_pipeline', ...), ('cat_pipeline', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,missing_values,nan
,strategy,'median'
,fill_value,None


In [ ]:
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

In [ ]:
X_train_processed = X_train_processed.astype('float')
X_test_processed = X_test_processed.astype('float')

# TensorFlow Sequential Model

In [ ]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train_processed.shape[1], )),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps = 1000,
    decay_rate = 0.96,
    staircase = True
)

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
    metrics=[
        tf.keras.metrics.Accuracy(name='accuracy'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.AUC(name='auc')])

model.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_69 (Dense)            (None, 64)                832       
                                                                 
 dropout_6 (Dropout)         (None, 64)                0         
                                                                 
 dense_70 (Dense)            (None, 32)                2080      
                                                                 
 dense_71 (Dense)            (None, 1)                 33        
                                                                 
Total params: 2,945
Trainable params: 2,945
Non-trainable params: 0
_________________________________________________________________


In [ ]:
import os
from datetime import datetime

log_dir = "logs/fit/" + datetime.now().strftime('%Y%m%d-%H%M%S')

callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath='myModel_{epoch:02d}.keras',
        save_best_only = True,
        monitor='val_loss',
        verbose=2),

    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        min_delta=1e-2,
        patience=5,
        verbose=2),

    tf.keras.callbacks.TensorBoard(
        log_dir=log_dir,
        histogram_freq=1)
]

model.fit(
    X_train_processed,
    y_train,
    epochs=5,
    batch_size=32,
    callbacks=callbacks,
    validation_split=0.2,
    verbose=2
)

Epoch 1/5

Epoch 1: val_loss improved from inf to 0.13518, saving model to myModel_01.keras
140/140 - 2s - loss: 0.3302 - accuracy: 0.0000e+00 - precision: 0.8524 - auc: 0.9491 - val_loss: 0.1352 - val_accuracy: 0.0000e+00 - val_precision: 0.8959 - val_auc: 0.9926 - 2s/epoch - 13ms/step
Epoch 2/5

Epoch 2: val_loss improved from 0.13518 to 0.08021, saving model to myModel_02.keras
140/140 - 0s - loss: 0.1251 - accuracy: 0.0000e+00 - precision: 0.9385 - auc: 0.9910 - val_loss: 0.0802 - val_accuracy: 0.0000e+00 - val_precision: 0.9681 - val_auc: 0.9970 - 471ms/epoch - 3ms/step
Epoch 3/5

Epoch 3: val_loss improved from 0.08021 to 0.06757, saving model to myModel_03.keras
140/140 - 0s - loss: 0.0927 - accuracy: 2.2321e-04 - precision: 0.9535 - auc: 0.9948 - val_loss: 0.0676 - val_accuracy: 0.0000e+00 - val_precision: 0.9657 - val_auc: 0.9978 - 491ms/epoch - 4ms/step
Epoch 4/5

Epoch 4: val_loss improved from 0.06757 to 0.05554, saving model to myModel_04.keras
140/140 - 0s - loss: 0.0799 

# Evaluation

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score

probs = model.predict(X_test_processed).flatten()
preds = (probs > 0.5).astype(int)

44/44 [==============================] - 0s 1ms/step


In [ ]:
print(f"Classification Report: {classification_report(y_test, preds)}")

Classification Report:               precision    recall  f1-score   support

           0       0.99      0.99      0.99       884
           1       0.98      0.98      0.98       516

    accuracy                           0.99      1400
   macro avg       0.98      0.99      0.99      1400
weighted avg       0.99      0.99      0.99      1400



In [ ]:
print(f"ROC AUC: {roc_auc_score(y_test, preds)}")

ROC AUC: 0.9860263425584903


# Feature Importance (Permutation Importance)

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.inspection import permutation_importance

class KerasWrapper(BaseEstimator, ClassifierMixin):
    def __init__(self, model):
        super().__init__()

        self.model = model

    def fit(self, X, y):
        return self


    def predict(self, X):
        # Returns hard labels (0 or 1)
        probs = self.model.predict(X, verbose=0).flatten()
        return (probs > 0.5).astype("int32")

    def predict_proba(self, X):
        # Returns probabilities (shape must be [n_samples, 2] for sklearn)
        prob_pos = self.model.predict(X, verbose=0).flatten()
        prob_neg = 1.0 - prob_pos

        # Stack them: Column 0 = Prob(Class 0), Column 1 = Prob(Class 1)
        return np.column_stack([prob_neg, prob_pos])

wrapped_model = KerasWrapper(model)

In [ ]:
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.base import is_classifier # Import check utility

# 1. Put ClassifierMixin FIRST
class KerasClassifierWrapper(ClassifierMixin, BaseEstimator):
    # 2. Define this at the CLASS level to satisfy all sklearn inspectors
    _estimator_type = "classifier"

    def __init__(self, model):
        self.model = model
        self.classes_ = np.array([0, 1])

    def fit(self, X, y):
        return self

    def predict(self, X):
        probs = self.model.predict(X, verbose=0).flatten()
        return (probs > 0.5).astype("int32")

    def predict_proba(self, X):
        prob_pos = self.model.predict(X, verbose=0).flatten()
        prob_neg = 1.0 - prob_pos
        return np.column_stack([prob_neg, prob_pos])

# --- Re-instantiate and Verify ---

wrapped_model = KerasClassifierWrapper(model)

# DEBUG: Verify sklearn sees it as a classifier BEFORE running the heavy task
print(f"Is classifier? {is_classifier(wrapped_model)}")
# This MUST print 'True'. If it prints 'False', the error will persist.

if is_classifier(wrapped_model):
    print("Starting permutation importance...")
    result = permutation_importance(
        estimator=wrapped_model,
        X=X_train_processed,
        y=y_train,
        n_repeats=5,
        scoring='roc_auc',
        random_state=42,
        n_jobs=1
    )

    print("Feature Importances computed successfully:")
    print(result.importances_mean)
else:
    print("STOP: Scikit-learn still thinks this is a Regressor. Check inheritance.")

Is classifier? True
Starting permutation importance...
Feature Importances computed successfully:
[1.66545951e-02 2.10650022e-01 4.04634126e-02 7.64595998e-05
 2.01217007e-01 1.65072136e-03 1.38592277e-03 2.07153702e-03
 1.74236871e-03 1.61272459e-03 7.82415553e-05 8.40808865e-05]
